Initial Imports:

In [10]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pandas as pd

1.***[Loading the data]***:
For MNIST:
In Python using TensorFlow and Keras:

In [3]:
# Transformations to apply to the dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the data
])

# Load the MNIST dataset
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:01<00:00, 5712550.90it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 68207034.81it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 12232656.54it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

100%|██████████| 4542/4542 [00:00<00:00, 967719.64it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw


For CIFAR-10:


In [4]:
# Transformations for CIFAR-10
transform_cifar = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the data
])

# Load the CIFAR-10 dataset
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)


100%|██████████| 170498071/170498071 [00:50<00:00, 3365940.64it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Paper results:

In [11]:
file_path = 'paper_results.csv'

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv(file_path)
print("Results obtained in the paper:")
data

Results obtained in the paper:


,Method,MNIST ACC,MNIST NMI,USPS ACC,USPS NMI,CIFAR-10 ACC,CIFAR-10 NMI
0,K-means,0.58,0.49,0.48,0.42,0.14,0.12
1,Deep Cluster,0.86,0.83,0.67,0.69,NA,NA
2,Deep K-means,0.84,0.80,0.76,0.78,NA,NA
3,CSC No Flatten,0.85,0.79,0.83,0.78,0.12,0.08
4,CSC No Filter,0.83,0.76,0.84,0.79,0.14,0.10
5,CSC No Voting,0.82,0.77,0.82,0.76,0.14,0.10
6,CSC,0.86,0.81,0.83,0.79,0.15,0.11


2.***[Building the model]***:
Autoencoder:

In [8]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

# Define the Convolutional Autoencoder model
class ConvAutoencoder(nn.Module):
    def __init__(self, input_channels):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, input_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define transformations and load datasets
transform = transforms.Compose([
    transforms.ToTensor(),
])



# Similarly, train the CIFAR-10 autoencoder


Files already downloaded and verified
Epoch [1/10], Loss: 0.0025
Epoch [2/10], Loss: 0.0011
Epoch [3/10], Loss: 0.0008
Epoch [4/10], Loss: 0.0006
Epoch [5/10], Loss: 0.0006
Epoch [6/10], Loss: 0.0005
Epoch [7/10], Loss: 0.0005
Epoch [8/10], Loss: 0.0004
Epoch [9/10], Loss: 0.0004
Epoch [10/10], Loss: 0.0004


# Load MNIST and CIFAR-10 datasets

In [ ]:
mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
cifar_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Set the data loader batch size

In [ ]:
batch_size = 64
mnist_train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=batch_size, shuffle=True)
cifar_train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size=batch_size, shuffle=True)


# Set the data loader batch size

In [ ]:
batch_size = 64
mnist_train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=batch_size, shuffle=True)
cifar_train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size=batch_size, shuffle=True)

# Initialize the autoencoder model

In [ ]:
input_channels_mnist = 1  # MNIST images are grayscale
input_channels_cifar = 3  # CIFAR-10 images have 3 channels (RGB)

mnist_autoencoder = ConvAutoencoder(input_channels_mnist)
cifar_autoencoder = ConvAutoencoder(input_channels_cifar)

# Loss function and optimizer

In [ ]:
criterion = nn.MSELoss()
optimizer_mnist = torch.optim.Adam(mnist_autoencoder.parameters(), lr=0.001)
optimizer_cifar = torch.optim.Adam(cifar_autoencoder.parameters(), lr=0.001)

**[Training for MNIST]**:

In [ ]:
for epoch in range(10):
    for data in mnist_train_loader:
        inputs, _ = data
        optimizer_mnist.zero_grad()
        outputs = mnist_autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer_mnist.step()
    print(f"Epoch [{epoch+1}/{10}], Loss: {loss.item():.4f}")